# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
#print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'
#print(filepath)

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))

    print(file_path_list[:4])

['/home/workspace/event_data/2018-11-27-events.csv', '/home/workspace/event_data/2018-11-04-events.csv', '/home/workspace/event_data/2018-11-07-events.csv', '/home/workspace/event_data/2018-11-09-events.csv']
['/home/workspace/event_data/.ipynb_checkpoints/2018-11-15-events-checkpoint.csv', '/home/workspace/event_data/.ipynb_checkpoints/2018-11-22-events-checkpoint.csv', '/home/workspace/event_data/.ipynb_checkpoints/2018-11-08-events-checkpoint.csv', '/home/workspace/event_data/.ipynb_checkpoints/2018-11-29-events-checkpoint.csv']


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
from time import time
start = time()

# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
# extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

times = time()-start
print(times)

0.08669185638427734


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


In [5]:
# Looking for structure of table
df = pd.read_csv('event_datafile_new.csv')
print(df.shape)
df.head(2)

(6820, 11)


,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Harmonia,Ryan,M,0,Smith,655.77751,free,"San Jose-Sunnyvale-Santa Clara, CA",583,Sehr kosmisch,26
1,The Prodigy,Ryan,M,1,Smith,260.07465,free,"San Jose-Sunnyvale-Santa Clara, CA",583,The Big Gundown,26


# Part II. Let's complete the Apache Cassandra coding portion. 

#### Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [6]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [7]:
# Create a Keyspace 

try:
    query = """CREATE KEYSPACE IF NOT EXISTS sparkify 
    WITH REPLICATION = {'class':'SimpleStrategy', 'replication_factor': 1}"""
    session.execute(query)
except Exception as e:
    print(e)

#### Set Keyspace

In [8]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)


## Create queries to ask the following three questions of the data

# Query 1
Give me the `artist`, `song` title and song's `length` in the **`music app history`** that was heard during `sessionId = 338`, and `itemInSession  = 4`

In [9]:
# Answer of Query 1
# Create Table music_app_history
try:
    create = "CREATE TABLE IF NOT EXISTS music_app_history"
    create = create + """(artist text, 
                            song text,
                            length float,
                            sessionId int,
                            itemInSession int,
                            PRIMARY KEY (sessionId, itemInSession))
                            """
    session.execute(create)

except Exception as e:
    
    print(e)

In [10]:
# Insert Table music_app_history
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        insert = "INSERT INTO music_app_history (artist, song, length, sessionId, itemInSession )"
        insert = insert + "VALUES (%s, %s, %s, %s,%s)" 
        ## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(insert, (line[0], line[9], float(line[5]), int(line[8]), int(line[3]) ))

#### SELECT to verify that the data have been inserted into each table

In [11]:
# Select from table music_app_history
## Add in the SELECT statement to verify the data was entered into the table
try:
    select = "SELECT artist, song, length FROM music_app_history WHERE sessionId = 338 AND itemInSession = 4"
    rows = session.execute(select)
    
    for row in rows:
        print(row.artist, row.song, row.length)
except Exception as e:
    print(e)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


# Query 2

Give me only the following: name of `artist`, `song `(sorted by `itemInSession`) and user (`first` and `last name`) for `userid = 10`, `sessionid = 182`

In [12]:
# Answer of Query 2
# Create Table music_app_history2
## Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)
## for userid = 10, sessionid = 182

try: 
    create = "CREATE TABLE IF NOT EXISTS user_session_history"
    create = create +  """(artist text,
                            song text,
                            firstName text,
                            lastName text,  
                            userid int, 
                            sessionId int, 
                            itemInsession int, 
                            PRIMARY KEY (userid, sessionid, itemInSession))"""

    session.execute(create)
    
except Exception as e:
    print(e)

In [13]:
# Insert Table music_app_history2
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        ## Assign the INSERT statements into the `query` variable
        insert = """INSERT INTO user_session_history (artist,firstname, lastname, userid, song, iteminsession, sessionid)
        VALUES (%s, %s, %s, %s, %s, %s, %s)"""

        session.execute(insert, (line[0], line[1], line[4], int(line[10]), line[9], int(line[3]), int(line[8])))    
                   

In [14]:
# Select Table music_app_history2
## Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)
## for userid = 10, sessionid = 182

try:
    select="""SELECT artist, firstname, lastname, song FROM user_session_history
    WHERE userid=10 and sessionid=182"""
    
    rows = session.execute(select)
    
    for row in rows:
        print(row.artist, row.firstname, row.lastname, row.song)
        
except Exception as e:
    print(e)
    

Down To The Bone Sylvie Cruz Keep On Keepin' On
Three Drives Sylvie Cruz Greece 2000
Sebastien Tellier Sylvie Cruz Kilometer
Lonnie Gordon Sylvie Cruz Catch You Baby (Steve Pitron & Max Sanna Radio Edit)


# Query 3
Give me every user name (`first` and `last name`) in my **`music app history`** who listened to the **`song 'All Hands Against His Own'`**

In [15]:
# Answer of Query 3
# Create Table music_app_history3
## Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
try:

    create = "CREATE TABLE IF NOT EXISTS user_song_history "
    create = create + """(firstname text,
                            lastname text,
                            song text,
                            userId int,
                            PRIMARY KEY (song, userId))"""

    session.execute(create)
    
except Exception as e:
    print(e)


In [16]:
# Insert Table music_app_history3
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    
    for line in csvreader:
        ## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO user_song_history (firstName, lastName, song, userId)"
        query = query + " VALUES (%s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (line[1], line[4], line[9], int(line[10])))

In [17]:
## Add in the SELECT statement to verify the data was entered into the table 

try:
    select = """SELECT firstname, lastname FROM user_song_history
    WHERE song='All Hands Against His Own'"""
    rows = session.execute(select)
    
    for row in rows:
        print (row.firstname, row.lastname)
    
except Exception as e:
    print(e)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [18]:
## Drop the table before closing out the sessions

try:
    session.execute("DROP TABLE IF EXISTS sparkify.music_app_history")
    session.execute("DROP TABLE IF EXISTS sparkify.user_session_history")
    session.execute("DROP TABLE IF EXISTS sparkify.user_song_history")
    print("Drop all Tables")
except Exception as e:
    print(e)

Drop all Tables


### Close the session and cluster connection¶

In [19]:
session.shutdown()
cluster.shutdown()